In [1]:
import torch
from pandas import read_csv
from torch.optim import SGD
from torch.nn import MSELoss
from torch.utils.data import DataLoader
from tensorboardX import SummaryWriter
from torch import randperm
from config import DATA_DIR, MODELS_DIR
from src.data_set import RatingsDataset
from src.model import MF
from src.runner import Runner
from src.utils import (
    create_dataset,
    mine_outliers_scipy,
    DataConverter,
)
from src.consistency import direct_calculation


"""
The Deam dataset is based on Arousal-Valence 2D emotional model.
The Valence/Arousal ratings were collected using Amazon Mechanical Turks service.
Each turk from the collected crowd were asked to mark his own emotion for the current song on a 2D plane, Arousal/Valence.
For more information please read: https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0173392
"""

DF_PATH = f"{DATA_DIR}" \
          f"/DEAM/annotations/annotations per each rater/" \
          f"song_level/static_annotations_songs_1_2000.csv"

def select_n_random(trainset: RatingsDataset):
    """
    Selects n random data points and their corresponding labels from a dataset
    """
    perm = randperm(len(trainset))
    return trainset[perm][:100]

In [2]:
"""
This block analyze raw data consistency using the direct calculation defined by:
consistency += row.rating - row.song.mean() for all rows in dataset
"""
columns = ["workerID", "SongId", "Valence"]
original_df = read_csv(DF_PATH, skipinitialspace=True, usecols=columns)
original_df.columns = ["user_id", "item_id", "rating"]
consistency = direct_calculation(data_frame=original_df)
print(f"Raw data consistency according to direct calculation is: \x1b[33m{consistency}\x1b[32m")

  0%|          | 0/17464 [00:00<?, ?it/s]

Raw data consistency according to direct calculation is: 2.5579538487363607e-13


In [3]:
"""
This block of code calculates the outliers alongside the valence axis
"""
columns = ["workerID", "SongId", "Valence"]
original_df = read_csv(DF_PATH, skipinitialspace=True, usecols=columns)
original_df.columns = ["user_id", "item_id", "rating"]

data_converter = DataConverter(
        original_df=original_df, n_random_users=0, n_ratings_per_random_user=9
)

valence_model = MF(
    n_users=data_converter.n_users,
    n_items=data_converter.n_item,
    include_bias=True
)
epochs = 100

criterion = MSELoss()
optimizer = SGD(valence_model.parameters(), lr=5, weight_decay=1e-3)
runner = Runner(
    model=valence_model,
    criterion=criterion,
    optimizer=optimizer,
    epochs=epochs
)

train_set = create_dataset(data_converter=data_converter)
train_load = DataLoader(train_set, batch_size=1000, shuffle=True)
users, items, ratings = select_n_random(train_set)

with SummaryWriter("runs/DEAM/valence") as writer:
    writer.add_graph(valence_model, (users, items))

    for epoch in range(epochs):
        epoch_loss = runner.train(train_loader=train_load, epoch=epoch, writer=writer)

        print(f"epoch={epoch + 1}, loss={epoch_loss}")

torch.save(valence_model.state_dict(), f"{MODELS_DIR}/DEAM/raw/valence.pt")

100%|██████████| 18/18 [00:00<00:00, 103.29batch/s, train_loss=0.0122] 


epoch=1, loss=0.17982098235755134


100%|██████████| 18/18 [00:00<00:00, 54.19batch/s, train_loss=0.00933]


epoch=2, loss=0.09419888197964636


100%|██████████| 18/18 [00:00<00:00, 67.95batch/s, train_loss=0.00823]


...

epoch=98, loss=0.010878291750776356


100%|██████████| 18/18 [00:00<00:00, 51.50batch/s, train_loss=0.0013]  


epoch=99, loss=0.010777630216088789


100%|██████████| 18/18 [00:00<00:00, 63.92batch/s, train_loss=0.00119] 

epoch=100, loss=0.010644557937465868


In [4]:
"""
This block of code calculates the outliers alongside the arousal axis
"""
columns = ["workerID", "SongId", "Arousal"]
original_df = read_csv(DF_PATH, skipinitialspace=True, usecols=columns)
original_df.columns = ["user_id", "item_id", "rating"]

data_converter = DataConverter(
        original_df=original_df, n_random_users=0, n_ratings_per_random_user=9
)

arousal_model = MF(
    n_users=data_converter.n_users,
    n_items=data_converter.n_item,
    include_bias=True
)
epochs = 100

criterion = MSELoss()
optimizer = SGD(arousal_model.parameters(), lr=5, weight_decay=1e-3)
runner = Runner(
    model=arousal_model,
    criterion=criterion,
    optimizer=optimizer,
    epochs=epochs,
)

train_set = create_dataset(data_converter=data_converter)
train_load = DataLoader(train_set, batch_size=1000, shuffle=True)
users, items, ratings = select_n_random(train_set)

with SummaryWriter("runs/DEAM/arousal") as writer:
    writer.add_graph(arousal_model, (users, items))

    for epoch in range(epochs):
        epoch_loss = runner.train(train_loader=train_load, epoch=epoch, writer=writer)
        print(f"epoch={epoch + 1}, loss={epoch_loss}")

torch.save(arousal_model.state_dict(), f"{MODELS_DIR}/DEAM/raw/arousal.pt")

100%|██████████| 18/18 [00:00<00:00, 79.73batch/s, train_loss=0.0135]


epoch=1, loss=0.17883310205360936


100%|██████████| 18/18 [00:00<00:00, 57.15batch/s, train_loss=0.01]   


epoch=2, loss=0.09467792048947565


100%|██████████| 18/18 [00:00<00:00, 83.23batch/s, train_loss=0.00665]


...

epoch=98, loss=0.010581601203515612


100%|██████████| 18/18 [00:00<00:00, 59.55batch/s, train_loss=0.000914]


epoch=99, loss=0.010408482466792239


100%|██████████| 18/18 [00:00<00:00, 45.81batch/s, train_loss=0.00158] 


epoch=100, loss=0.010911776545746573


In [5]:
valence_outliers = mine_outliers_scipy(model=valence_model, data_converter=data_converter)
arousal_outliers = mine_outliers_scipy(model=arousal_model, data_converter=data_converter)

items_group_by_users = data_converter.original_df.groupby("user_id")
combined_outliers = {}
for user_id, valence_dist in valence_outliers.items():
    arousal_dist = arousal_outliers[user_id]
    combined_outliers[user_id] = valence_dist + arousal_dist

combined_outliers = dict(sorted(combined_outliers.items(), key=lambda item: item[1]))
for user_id, item_id in combined_outliers.items():
    number_of_items = len(items_group_by_users.get_group(user_id))
    print(f"user: {user_id}, dist: {item_id}, #items: {number_of_items}")

user: 2a6b63b7690efa2390c8d9fee11b1407, dist: -24.267710561123145, #items: 3
user: ad3b997c4f2382a66e49f035cacfa682, dist: -12.296149919475926, #items: 3
user: 65794ea9f5122952403585a237bc5e52, dist: 3.9826956566666603, #items: 3
user: 374a5659c02e12b01db6319436f17a7d, dist: 9.869268496983022, #items: 3
user: 615d836ba25132081e0ebd2182221a59, dist: 11.557994451083452, #items: 6
user: 623681f76a3eab5d9c86fbc0e1ca264b, dist: 12.954068824121107, #items: 12
user: fd5b08ce362d855ca9152a894348130c, dist: 15.645982217473833, #items: 222
user: da37d1548ffd0631809f7be341e4fe4d, dist: 20.323636435677617, #items: 3
user: a30d244141cb2f51e0803e79bc4bd147, dist: 23.57655772998743, #items: 985
user: 6222da90667e5b0de990ce6c26dcfa15, dist: 25.76786325272345, #items: 12
user: 46a2289decf79f747406fa91cd92fc27, dist: 29.46003879800084, #items: 333
user: 00de940f0b5cfc82cca4791199e3bfb3, dist: 30.80378344960348, #items: 751
user: 485d8e33a731a830ef0aebd71b016d08, dist: 33.635854594780135, #items: 6
user: